# Capstone Project - **Coffee shop** in **Tirana**
### Applied Data Science Capstone by IBM/Coursera
#### Final Assignment

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)


## Introduction: Business Problem <a name="introduction"></a>


This project wants to show if it is convenient to open a **Coffee shop** in **Tirana**, Albania. People who might be interested in this analysis are future young entrepreneurs, who are new to the Coffee shop industry.

Tirana has a population density of 500 people per square kilometre, is the most densely populated municipality in the country. About 1/4 of the country's total population is located in Tirana.
Now, since the citizens of Tirana like a good coffee in company, it may be a good idea to open a Coffee shop, let find it out, if it really is so!

We will try to detect **locations that are not already crowded with Coffee shops**. We are interested in locations with **no Coffee shops in nearby** and for sure **as close to city center as possible**, assuming that first two conditions are met.

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>


Based on definition of our problem, factors that will influence our decission are:
* number of existing Coffe shops in the neighborhood
* distance to Coffe shops in the neighborhood, if any:
* distance of neighborhood from city center

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Google Maps API reverse geocoding**
* number of Coffe shops and their type and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of Tirana city center will be obtained using **Google Maps API geocoding** of the known Skanderbeg Square.


##### TIRANA CITY CENTER

Let's first find the latitude & longitude of Tirana city center, using specific, well known address and Google Maps geocoding API.

###### First we import all the Libraries we need:

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
import sys
from bs4 import BeautifulSoup
import os
from geopy.geocoders import Nominatim 
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline


print('Libraries imported.')

Libraries imported.


In [2]:
!pip install geocoder
!pip install folium

In [3]:
import json
url = 'https://api.foursquare.com/v2/venues/explore'

params = dict(
client_id='UR0UP5N0SGHDMNDAKQHZG213Y1ESEGPFOZ2OZHKSBRZQQ4ZM',
client_secret='GDFCMZSWXMJZUOAEX3ERJVLHHQQFIL4ZPTINJIWWJZCVLVYP',
v='20201119',
ll='41.328868,19.818570',
query='coffee',
limit=1
)
resp = requests.get(url=url, params=params)
data = json.loads(resp.text)
print (data)

{'meta': {'code': 200, 'requestId': '5fdca819bec12724ec5f41ae'}, 'response': {'suggestedFilters': {'header': 'Tap to show:', 'filters': [{'name': 'Open now', 'key': 'openNow'}]}, 'warning': {'text': 'There aren\'t a lot of results for "coffee." Try something more general, reset your filters, or expand the search area.'}, 'suggestedRadius': 1618, 'headerLocation': 'Tirana', 'headerFullLocation': 'Tirana', 'headerLocationGranularity': 'city', 'query': 'coffee', 'totalResults': 72, 'suggestedBounds': {'ne': {'lat': 41.33217536675265, 'lng': 19.81920277856529}, 'sw': {'lat': 41.3294757069672, 'lng': 19.822262660200614}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '562a5fef498e8e32d797c9c2', 'name': 'Mulliri Vjeter', 'location': {'address': 'Rr. Barrikadave', 'lat': 41.330825536859926, 'lng': 19.82073271938295, 

In [4]:
CLIENT_ID = 'UR0UP5N0SGHDMNDAKQHZG213Y1ESEGPFOZ2OZHKSBRZQQ4ZM' # your Foursquare ID
CLIENT_SECRET = 'GDFCMZSWXMJZUOAEX3ERJVLHHQQFIL4ZPTINJIWWJZCVLVYP' # your Foursquare Secret
VERSION = '20201119' # Foursquare API version
LIMIT = 15000 # A default Foursquare API limit value
radius = 10000

In [5]:
# Coorinates for Connaught Place abbreviated as CP
CP_latitude = 41.328868
CP_longitude = 19.818570

In [6]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    CP_latitude, 
    CP_longitude, 
    radius, 
    LIMIT,
)

##### Now we call for all the venues nearby the city center of Tirana

In [7]:
results = requests.get(url).json()
results.values();
results

{'meta': {'code': 200, 'requestId': '5fdca819dd4fd414b143a23f'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Tirana',
  'headerFullLocation': 'Tirana',
  'headerLocationGranularity': 'city',
  'totalResults': 76,
  'suggestedBounds': {'ne': {'lat': 41.41886809000009,
    'lng': 19.938197488412005},
   'sw': {'lat': 41.23886790999991, 'lng': 19.698942511587997}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5726844a38fa6e0abbf079c4',
       'name': 'The Plaza',
       'location': {'address': 'Rruga 28 Nëntori',
        'lat': 41.327662102745755,
        'lng': 19.82158864976927,
        'labeledLatLngs': [{'label': 'display',
          'lat': 41.327662102745755,
          'ln

In [8]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [9]:
from pandas.io.json import json_normalize

In [10]:
venues = results['response']['groups'][0]['items']
venues_neighborhood = [results['response']['groups'][0]['items'][n]['venue']['location']['formattedAddress'] for n in range(len(venues))]  
venues_distance = [results['response']['groups'][0]['items'][n]['venue']['location']['distance'] for n in range(len(venues))]  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.id','venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

#filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

[venues_neighborhood[k][0] for k in range(len(venues_neighborhood))]

#nearby_venues['neighborhood'] = venues_neighborhood[0][0]
nearby_venues['neighborhood'] =  [venues_neighborhood[k][0] for k in range(len(venues_neighborhood))]
nearby_venues['distance [m]'] = venues_distance

df = nearby_venues
df

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,lat,lng,neighborhood,distance [m]
0,5726844a38fa6e0abbf079c4,The Plaza,Hotel,41.327662,19.821589,Rruga 28 Nëntori,285
1,57ef7194498ed51bfd8a11bb,Artigiano at Vila,Italian Restaurant,41.322315,19.822859,Papa Gjon Pali II (Mustafa Matohiti),812
2,51608e36e4b0e63c74c2f869,Hemingway,Bar,41.329564,19.812277,Rr Mihal Duri,531
3,56ed51f5498e4ff4a52beaf4,Fishop,Seafood Restaurant,41.322384,19.827218,Rruga Qemal Guranjaku,1021
4,52acb96e11d2bfcca236c92f,à la Santé,French Restaurant,41.320760,19.814221,Rr. Sami Frasheri,973
5,56d0448acd10af7327eada1a,Sophie Caffe & Snacks Rr.kavajes,Café,41.328271,19.813140,Rr. Kavajes,458
6,55f9a548498e452a8f3942e5,Cioccolatitaliani,Ice Cream Shop,41.320285,19.818315,Shqipëria,955
7,506c8769e4b068c766393a00,Yogurteria,Ice Cream Shop,41.319718,19.818552,Blloku,1018
8,4c8bfb09509e370431322f55,Radio,Cocktail Bar,41.320205,19.815001,Rruga Ismail Qemali,1009
9,4e0af78b52b1fe83d548a3a3,Artigiano,Italian Restaurant,41.319398,19.818842,Rruga Abdyl Frasheri (Rruga Ibrahim Rugova),1054


In [11]:
nearby_venues.shape

(76, 7)

In [12]:
nearby_venues.columns

Index(['id', 'name', 'categories', 'lat', 'lng', 'neighborhood',
       'distance [m]'],
      dtype='object')

In [13]:
import pandas as pd
data = nearby_venues
nearby_venues = pd.DataFrame(data)

In [14]:
nearby_venues.head(10)

,id,name,categories,lat,lng,neighborhood,distance [m]
0,5726844a38fa6e0abbf079c4,The Plaza,Hotel,41.327662,19.821589,Rruga 28 Nëntori,285
1,57ef7194498ed51bfd8a11bb,Artigiano at Vila,Italian Restaurant,41.322315,19.822859,Papa Gjon Pali II (Mustafa Matohiti),812
2,51608e36e4b0e63c74c2f869,Hemingway,Bar,41.329564,19.812277,Rr Mihal Duri,531
3,56ed51f5498e4ff4a52beaf4,Fishop,Seafood Restaurant,41.322384,19.827218,Rruga Qemal Guranjaku,1021
4,52acb96e11d2bfcca236c92f,à la Santé,French Restaurant,41.320760,19.814221,Rr. Sami Frasheri,973
5,56d0448acd10af7327eada1a,Sophie Caffe & Snacks Rr.kavajes,Café,41.328271,19.813140,Rr. Kavajes,458
6,55f9a548498e452a8f3942e5,Cioccolatitaliani,Ice Cream Shop,41.320285,19.818315,Shqipëria,955
7,506c8769e4b068c766393a00,Yogurteria,Ice Cream Shop,41.319718,19.818552,Blloku,1018
8,4c8bfb09509e370431322f55,Radio,Cocktail Bar,41.320205,19.815001,Rruga Ismail Qemali,1009
9,4e0af78b52b1fe83d548a3a3,Artigiano,Italian Restaurant,41.319398,19.818842,Rruga Abdyl Frasheri (Rruga Ibrahim Rugova),1054


In [15]:
cafe_citycenter =  nearby_venues['categories']=='Café'
cafe_citycenter.value_counts()

False    68
True      8
Name: categories, dtype: int64

In [16]:
bar_citycenter =  nearby_venues['categories']=='Bar'
bar_citycenter.value_counts()

False    73
True      3
Name: categories, dtype: int64

In [17]:
coffeeshop_citycenter =  nearby_venues['categories']=='Coffee Shop'
coffeeshop_citycenter.value_counts()

False    75
True      1
Name: categories, dtype: int64

## Methodology <a name="data"></a>


In this project we will direct our efforts on deciding if it is convenient to open a Coffee shop in Tirana.
We have collected the required data: location and type (category) of every venue nearby Tirana center (Sheshi Skanderbeg).
We have also identified all the Bar, Café and Coffee Shop nearby the center (according to Foursquare categorization), in particular:
* number of existing Coffe shops, Bar and Café in the nearby city center of Tirana

## Analysis <a name="data"></a>

According to the data, near the city center there are:

* 8 Café
* 3 Bar 
* 1 Coffee Shop

So in total **12** places that can offer a good caffe in the center of the city. 

## Conclusion <a name="data"></a>

Taking into account that in the city center of Tirana everyday there are near **350.000-400.000** people circulating for work and leisure, i am seeing it as convenient to aopen a new good **Coffee shop** in **Tirana Center**.
